In [ ]:
! pip3 install langchain openai huggingface_hub python-dotenv chromadb langchain_openai langchain_chroma nltk evaluate rouge_score chroma-migrate PyPDF2 docx

Importing Libraries

In [ ]:

import numpy as np
import pandas as pd
import os
import openai
import sys
import io
import nltk
from evaluate import load
from nltk import word_tokenize
import evaluate
from PIL import Image
from langchain_chroma import Chroma
from huggingface_hub import hf_hub_download
from langchain_openai import OpenAIEmbeddings
from more_itertools import chunked
from transformers import BlipProcessor, BlipForConditionalGeneration
from nltk.translate.bleu_score import sentence_bleu

#Langchain related libraries
from langchain.schema import Document
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
#from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
import os

from chromadb.config import Settings
import chromadb
from chromadb.config import Settings
import warnings
warnings.filterwarnings("ignore")

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = 'Enter API'
os.environ["OPENAI_API_KEY"] ='Enter API'  # Add your api key here
openai.api_key  = OPENAI_API_KEY

nltk.download('punkt')
rouge = evaluate.load('rouge')

/home/varsh/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-07 12:37:32.108347: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-07 12:37:32.116353: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733535452.129500   27510 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733535452.133228   27510 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-07 12:37:32.148195: I tensor

In [7]:
print(os.listdir())
print(os.getcwd())

['USA', 'rag.ipynb', 'Europe', 'Australia', 'UK', 'India']
/home/varsh/Documents/UNSW/Hackathons/Tutero/Curriculum


In [8]:
countries = [d for d in os.listdir('.') if os.path.isdir(d)]
print(countries)

['USA', 'Europe', 'Australia', 'UK', 'India']


In [ ]:
import PyPDF2
embeddings = OpenAIEmbeddings()

In [ ]:
pdf_extensions = ('.pdf', )

chunk_size = 1000   
chunk_overlap = 200
batch_size = 50     
persist_base = "chroma_dbs"  
os.makedirs(persist_base, exist_ok=True)

for country in countries:
    print(f"Processing {country}...")
    country_dir = os.path.join('.', country)

    country_files = [
        f for f in os.listdir(country_dir)
        if f.lower().endswith(pdf_extensions)
    ]
    
    documents = []
    for file in country_files:
        file_path = os.path.join(country_dir, file)
        try:
            with open(file_path, 'rb') as pdf_file:
                reader = PyPDF2.PdfReader(pdf_file)
                page_texts = []
                for page in reader.pages:
                    if page is not None:
                        extracted = page.extract_text()
                        if extracted:
                            page_texts.append(extracted)
                text = "\n".join(page_texts)
        except Exception as e:
            print(f"Error reading {file}: {e}")
            continue

        if not text.strip():
            print(f"No text found in {file}, skipping.")
            continue

        text_splitter = CharacterTextSplitter(
            separator="\n",
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len
        )
        chunks = text_splitter.split_text(text)

        for chunk in chunks:
            documents.append(
                Document(
                    page_content=chunk,
                    metadata={"source": file, "country": country}
                )
            )

    if not documents:
        print(f"No valid documents found for {country}.")
        continue

    persist_dir = os.path.join(persist_base, f"chroma_{country}")
    os.makedirs(persist_dir, exist_ok=True)

    db = None
    total_docs = len(documents)

    for i in range(0, total_docs, batch_size):
        batch = documents[i:i+batch_size]
        try:
            if db is None:
                db = Chroma.from_documents(
                    documents=batch,
                    embedding=embeddings,
                    persist_directory=persist_dir
                )
            else:
                texts = [doc.page_content for doc in batch]
                metas = [doc.metadata for doc in batch]
                db.add_texts(texts, metadatas=metas)
            
            db.persist()
            print(f"Processed batch {i//batch_size+1} for {country}, {len(batch)} docs.")
        except Exception as e:
            print(f"Error processing batch for {country}: {e}")
            break

    print(f"Created embeddings for {country} with {total_docs} chunks total.")

print("Processing complete!")


Processing USA...


Created a chunk of size 1652, which is longer than the specified 1000
Created a chunk of size 1181, which is longer than the specified 1000
Created a chunk of size 1103, which is longer than the specified 1000
Created a chunk of size 1121, which is longer than the specified 1000
Created a chunk of size 1210, which is longer than the specified 1000
Created a chunk of size 1053, which is longer than the specified 1000
Created a chunk of size 1014, which is longer than the specified 1000
Created a chunk of size 1307, which is longer than the specified 1000
Created a chunk of size 1371, which is longer than the specified 1000
Created a chunk of size 1163, which is longer than the specified 1000
Created a chunk of size 1249, which is longer than the specified 1000
Created a chunk of size 1065, which is longer than the specified 1000
Created a chunk of size 1127, which is longer than the specified 1000
Created a chunk of size 1237, which is longer than the specified 1000


Processed batch 1 for USA, 50 docs.
Processed batch 2 for USA, 50 docs.
Processed batch 3 for USA, 50 docs.
Processed batch 4 for USA, 50 docs.
Processed batch 5 for USA, 50 docs.
Processed batch 6 for USA, 50 docs.
Processed batch 7 for USA, 50 docs.
Processed batch 8 for USA, 50 docs.
Processed batch 9 for USA, 50 docs.
Processed batch 10 for USA, 50 docs.
Processed batch 11 for USA, 50 docs.
Processed batch 12 for USA, 50 docs.
Processed batch 13 for USA, 35 docs.
Created embeddings for USA with 635 chunks total.
Processing Europe...
Processed batch 1 for Europe, 34 docs.
Created embeddings for Europe with 34 chunks total.
Processing Australia...


Created a chunk of size 1218, which is longer than the specified 1000


Processed batch 1 for Australia, 50 docs.
Processed batch 2 for Australia, 50 docs.
Processed batch 3 for Australia, 50 docs.
Processed batch 4 for Australia, 50 docs.
Processed batch 5 for Australia, 50 docs.
Processed batch 6 for Australia, 50 docs.
Processed batch 7 for Australia, 50 docs.
Processed batch 8 for Australia, 50 docs.
Processed batch 9 for Australia, 50 docs.
Processed batch 10 for Australia, 50 docs.
Processed batch 11 for Australia, 50 docs.
Processed batch 12 for Australia, 50 docs.
Processed batch 13 for Australia, 50 docs.
Processed batch 14 for Australia, 50 docs.
Processed batch 15 for Australia, 50 docs.
Processed batch 16 for Australia, 50 docs.
Processed batch 17 for Australia, 50 docs.
Processed batch 18 for Australia, 50 docs.
Processed batch 19 for Australia, 50 docs.
Processed batch 20 for Australia, 50 docs.
Processed batch 21 for Australia, 50 docs.
Processed batch 22 for Australia, 50 docs.
Processed batch 23 for Australia, 50 docs.
Processed batch 24 f

...

Processed batch 242 for Australia, 50 docs.


Exception occurred invoking consumer for subscription 114ef40de6fc4cf1b77c034586831c51to topic persistent://default/default/c766d7c5-1c1d-484a-888e-5d7d5b955e8a 'utf-8' codec can't encode character '\udd80' in position 31: surrogates not allowed


Processed batch 243 for Australia, 50 docs.


Exception occurred invoking consumer for subscription 114ef40de6fc4cf1b77c034586831c51to topic persistent://default/default/c766d7c5-1c1d-484a-888e-5d7d5b955e8a 'utf-8' codec can't encode character '\udd80' in position 12: surrogates not allowed


Processed batch 244 for Australia, 50 docs.
Processed batch 245 for Australia, 50 docs.
Processed batch 246 for Australia, 50 docs.
Processed batch 247 for Australia, 50 docs.
Processed batch 248 for Australia, 50 docs.
Processed batch 249 for Australia, 50 docs.
Processed batch 250 for Australia, 50 docs.
Processed batch 251 for Australia, 50 docs.
Processed batch 252 for Australia, 50 docs.
Processed batch 253 for Australia, 50 docs.
Processed batch 254 for Australia, 50 docs.
Processed batch 255 for Australia, 50 docs.
Processed batch 256 for Australia, 50 docs.
Processed batch 257 for Australia, 50 docs.
Processed batch 258 for Australia, 50 docs.
Processed batch 259 for Australia, 50 docs.
Processed batch 260 for Australia, 50 docs.
Processed batch 261 for Australia, 50 docs.
Processed batch 262 for Australia, 50 docs.
Processed batch 263 for Australia, 50 docs.
Processed batch 264 for Australia, 50 docs.
Processed batch 265 for Australia, 50 docs.
Processed batch 266 for Australi

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

db = Chroma(
    embedding_function=embeddings,
    persist_directory="chroma_dbs/chroma_Australia"
)

docs = db.similarity_search("Year 8 Science curriculum", k=4)
for d in docs:
    print(d.page_content, d.metadata)


a t t i t u d e s  t o w a r d s  d i ve r si t y  a n d   d i f f e r e n ce   a f f e ct   co m m u n i ca t i o n   a n d   t h e   su st a i n a b i l i t y  o f   co m m u n i t i e s
e xa m i n i n g   cu l t u r a l l y  sp e ci f i c  t e r m s  a n d   p h r a se s,   su ch   a s η παρέα , το κέφι, Στην υγειά σου, Με γεια, Γεια
στα χέρια σου/σας ,   d e ve l o p i n g   a p p r o p r i a t e   e xp l a n a t i o n s  f o r   t h e m   a n d   d i scu ssi n g   e q u i va l e n t   t e r m s  i n
o t h e r   l a n g u a g e s
Years 7–10 (Year 7 Entry) SequenceThe Australian Curriculum is licensed under Creative Commons.
Modern Greek
Years 7–10 (Year 7 Entry) Sequence Modern Greek
ACARA | The Australian Curriculum (Version 8.4)
The Australian Curriculum is licensed under Creative Commons.
Page 2511 of 3484
Y e a r s   7   a n d   8   A c h i e v e m e n t   S t a n d a r d s {'country': 'Australia', 'source': 'AustralianCurriculum (2).pdf'}
w e l l   a s  d e ci m a l s,   co m 

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0.7, model_name="gpt-4")

curriculum_context = "\n\n".join([d.page_content for d in docs])

LESSON_PLAN_TEMPLATE = """
You are a teaching assistant AI. Given the following curriculum context:

{curriculum_context}

Create a fully personalized lesson plan for a {grade_level}-year-old student in the {country} curriculum, focusing on {subject}. 

The lesson plan should include:
1. Title and learning objectives aligned with the curriculum.
2. A warm-up activity.
3. Theory explanation, age-appropriate and engaging.
4. Practice questions (mild, hot, spicy) with model answers.
5. Gamification ideas.
6. Resources (videos, worksheets, websites).
7. Differentiation (support for struggling students, extension for advanced).
8. A tutor reflection prompt at the end.

Now produce the lesson plan.
"""

prompt = PromptTemplate(
    input_variables=["curriculum_context", "grade_level", "country", "subject"],
    template=LESSON_PLAN_TEMPLATE
)

chain = LLMChain(llm=llm, prompt=prompt)

lesson_plan = chain.run(
    curriculum_context=curriculum_context,
    grade_level="14",      
    country="Australia",   
    subject="Science"     
)

print(lesson_plan)


Lesson Title: "The Thrilling World of Cells!"

Learning Objectives:
1. Understand the basic structure of a cell and its components.
2. Compare and contrast animal and plant cells.
3. Understand the role of cells in living organisms.
4. Develop scientific inquiry skills.

Warm-Up Activity:
Begin the lesson with a short quiz game called "Cell Jeopardy". The students will answer questions related to basic biology and cells to refresh their prior knowledge and prepare them for the lesson ahead.

Theory Explanation:
Using an interactive slideshow presentation, explain the structure of a cell, the function of each of its parts, and the differences between animal and plant cells. Use analogies and everyday examples to make the theory more relatable and engaging.

Practice Questions:

Mild: Label the parts of a plant and an animal cell (Worksheet provided).

Hot: Explain the function of each part of an animal and a plant cell in your own words.

Spicy: Compare and contrast the structure and fu